In [1]:
from collections import Counter
import itertools

import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import linalg 
from scipy.sparse import dok_matrix
from scipy.sparse import csr_array
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from itertools import zip_longest

df = pd.read_csv('../input/abcnews-date-text.csv')
nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))
df['tokenized'] = df.headline_text.str.split(' ')

df['length'] = df.tokenized.map(len)
df = df.loc[df.length > 1]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#explode and sanitize

ex = df.explode('tokenized')

ex = ex.loc[ex.tokenized.str.len() > 2]

ex = ex.loc[~ex.tokenized.isin(stopwords_set)]

In [3]:
#ceate dictionary of words

dictionary = ex.tokenized.drop_duplicates()

dictionary = pd.Series(dictionary.tolist(), name='words')

dictionary = dictionary.to_frame()

dictionary_lookup = dictionary.to_dict()['words']

dictionary['encode'] = dictionary.index

dictionary = dictionary.set_index('words')

dictionary.encode = dictionary.encode
dictionary = dictionary.to_dict()['encode']


In [4]:

encode = ex.tokenized.map(dictionary.get).to_frame()

encode

,tokenized
0,0
0,1
0,2
0,3
0,4
...,...
1244183,1319
1244183,112468
1244183,6182
1244183,450


In [5]:
#alternative
#np.split(a[:, 1], np.unique(a[:, 0], return_index=True)[1][1:])

encode.index.astype('int')
encode.tokenized.astype('int')
docs = encode.groupby(encode.index)['tokenized'].apply(list)



In [6]:

encoded_docs = list(zip(*zip_longest(*docs.to_list(), fillvalue=0)))

encoded_docs = csr_array(encoded_docs, dtype=int)

encoded_docs 

<1242915x12 sparse array of type '<class 'numpy.int64'>'
	with 6584619 stored elements in Compressed Sparse Row format>

In [19]:
from scipy.sparse import find

row_column_code = find(encoded_docs)

word_sorted_index = row_column_code[2].argsort()

doc_word = np.array([row_column_code[0][word_sorted_index], row_column_code[2][word_sorted_index]])

doc_word_sorted_index = doc_word[0].argsort()

doc_word = pd.DataFrame(np.array([doc_word[0][doc_word_sorted_index], doc_word[1][doc_word_sorted_index]]).T, colums=['doc','word'])

doc_word



,0,1
0,0,1
1,0,2
2,0,3
3,0,4
4,1,7
...,...,...
6584614,1242914,677
6584615,1242914,899
6584616,1242914,112468
6584617,1242914,450


In [20]:
doc_word.groupby([0,1]).count()



Empty DataFrame
Columns: []
Index: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 11), (2, 12), (2, 13), (2, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (4, 15), (4, 18), (4, 21), (4, 22), (4, 23), (5, 24), (5, 25), (5, 26), (5, 27), (5, 28), (6, 29), (6, 30), (6, 31), (6, 32), (6, 33), (7, 34), (7, 35), (7, 36), (7, 37), (7, 38), (7, 39), (7, 40), (8, 17), (8, 41), (8, 42), (8, 43), (8, 44), (9, 45), (9, 46), (9, 47), (9, 48), (9, 49), (10, 44), (10, 45), (10, 50), (10, 51), (10, 52), (11, 31), (11, 33), (11, 53), (11, 54), (11, 55), (11, 56), (12, 57), (12, 58), (12, 59), (12, 60), (13, 61), (13, 62), (13, 63), (13, 64), (13, 65), (14, 61), (14, 66), (14, 67), (14, 68), (14, 69), (14, 70), (15, 71), (15, 72), (15, 73), (15, 74), (15, 75), (16, 76), (16, 77), (16, 78), (16, 79), (16, 80), (17, 79), (17, 81), (17, 82), (17, 83), (17, 84), (17, 85), (18, 86), (18, 87), (18, 88), (18, 89), (18, 90), (18, 91), ...]

[6575726 rows x 0 columns]